<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Function-definitions" data-toc-modified-id="Function-definitions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function definitions</a></span></li><li><span><a href="#Open-Serial-Port" data-toc-modified-id="Open-Serial-Port-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Open Serial Port</a></span><ul class="toc-item"><li><span><a href="#Loop-sending-bytes" data-toc-modified-id="Loop-sending-bytes-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Loop sending bytes</a></span></li><li><span><a href="#Send-bytes-in-HDLC-in-a-loop" data-toc-modified-id="Send-bytes-in-HDLC-in-a-loop-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Send bytes in HDLC in a loop</a></span></li><li><span><a href="#Send-sine-signal-in-a-Loop" data-toc-modified-id="Send-sine-signal-in-a-Loop-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Send sine signal in a Loop</a></span></li></ul></li><li><span><a href="#Open-wav-file" data-toc-modified-id="Open-wav-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Open wav file</a></span><ul class="toc-item"><li><span><a href="#Just-play-the-song-direcly-in-Python" data-toc-modified-id="Just-play-the-song-direcly-in-Python-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Just play the song direcly in Python</a></span></li><li><span><a href="#Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python" data-toc-modified-id="Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Send Data of song via UART to PC and play it in Python</a></span></li><li><span><a href="#Send-bytes-of-wav-and-receive-aknowledge" data-toc-modified-id="Send-bytes-of-wav-and-receive-aknowledge-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Send bytes of wav and receive aknowledge</a></span></li></ul></li><li><span><a href="#Get-decoded-mp3-data" data-toc-modified-id="Get-decoded-mp3-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get decoded mp3 data</a></span></li><li><span><a href="#Check-consistency-of-sent-data" data-toc-modified-id="Check-consistency-of-sent-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Check consistency of sent data</a></span></li><li><span><a href="#Debug" data-toc-modified-id="Debug-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Debug</a></span></li></ul></div>

# Imports

In [1]:
import serial
import time
from pathlib import Path
import wave
import pyaudio
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as pl

In [ ]:
%matplotlib qt

# Function definitions

In [2]:
def array_to_big_endian_bytes(array):
    return bytes(array.astype('>i2'))

In [3]:
def create_hdlc_frame(data):
    """
    Creates a hdlc frame from bytearray (adds frame delimiters(0x7E) and escapes flags(0x7E, 0x7D) payload)
    
    Parameter:
    ----------
    
    data (bytearray):
        payload data
        
    Returns:
    -------
    
    frame (bytearray):
        payload embedded in hdlc frame
    """
    if type(data) != bytes:
        msg = "Unexpected data type. Received {}, but expected 'bytes'".format(type(data))
        raise TypeError(msg)

    if (len(data) % 2) != 0:
        raise ValueError("Array length is not even!")
    
    frame = b'\x7E' + \
            data.replace(b'\x7D', b'\x7D\x5D') \
                .replace(b'\x7E', b'\x7D\x5E') \
            + b'\x7E'
            
    return frame

In [4]:
def get_transmission_chunk(wav, chunk_size, shift = 0):
    frame = create_hdlc_frame((wav.readframes(chunk_size))>>shift)
    return frame

In [5]:
def attenuate_audio(wav, divisor=16):
    raw_data = wav.readframes(wav.getnframes())
    dt = np.dtype(np.int16) #.newbyteorder('>')
    raw_data = np.frombuffer(raw_data, dtype=dt)//divisor
    return raw_data

In [6]:
def comp_send_receive(rec_data, sent_data):
    """
    Compare array rec_data with array sent_data.
    rec_data is expected to consist of n times sent_data, where the first and
    last occurence of sent_data might be incomplete.

    Parameter:
    ----------

    rec_data (np.array):
        Array which contains n-times sent_data,  where the first and
        last occurence of sent_data might be incomplete.
    
    sent_data (np.array):
        Array, which is the "element" of received_array

    Return:
    -------
        True:   rec_data consists exclusivly of several occurences of sent_data
        False:  -"- vice versa
    """
    offs = np.where(sent_data == rec_data[0])[0][0]
    frame_len = len(sent_data)
    len_start_frame = frame_len-offs

    num_recvd_frames, len_end_frame = divmod(len(rec_data)-len_start_frame, frame_len)

    equal = np.array_equal(rec_data[0:len_start_frame], sent_data[offs:])
    for i in range(0, num_recvd_frames):
        equal &= np.array_equal(rec_data[len_start_frame + i*frame_len: len_start_frame +i *frame_len + frame_len], sent_data)
    if len_end_frame != 0:
        equal &= np.array_equal(rec_data[len(rec_data)-len_end_frame:], sent_data[0:len_end_frame])

    return equal

In [7]:
def create_sine_samples(amplitude, frequency, samplerate):
    """
    Creates an array containing one sine period with defined frequency and amplitude strobed by the samplerate.
    Output is casted to int16
    
    Parameter:
    ----------
    
    amplitude (number):
        desired amplitude
        
    frequency (number):
        desired frequency in Hz
        
    samplerate (number):
        desired frequnecy in Hz
        
    Returns:
    -------
    
    samples (np.array of np.int16):
        mp.array containing sine samples (one period)
    """
    step_size = 2 * math.pi * frequency / samplerate
    samples = [np.int16(amplitude * math.sin(sample)) for sample in np.arange(0, 2*math.pi, step_size)]
    return samples

# Open Serial Port

In [8]:
ser = serial.Serial('COM5', 2e6, timeout=1, parity=serial.PARITY_NONE) # the baudrate of the uart dongle in verry unprecise. 1.5MHz is matched verry good.

In [ ]:
ser.close()

## Loop sending bytes

In [ ]:
while(True):
    for byte in range(0,0x100,1):
        #print(byte)
        ser.write(bytes([byte])) #, 0x01, 0x7F, 0x3F, 0x0F,0xFF]))
        time.sleep(1e-2)

In [ ]:
ser.write(bytes([0x7E, 0x77, 0xD7]))
ser.write(bytes([0x7E, 0x55, 0xD8]))
ser.write(bytes([0x7E, 0xAA, 0xD9]))

In [ ]:
ser.read_all()

In [ ]:
while(True):
    time.sleep(1)
    ser.write(bytes([0x00, 0x77, 0x55]))

## Send bytes in HDLC in a loop

In [ ]:
data_little_endian = (np.arange(start=0x30, stop=0x40, step=1,dtype=np.int16()))

In [ ]:
data_big_endian = array_to_big_endian_bytes(data_little_endian)
hdlc_frame = create_hdlc_frame(data=data_big_endian)

In [ ]:
hdlc_frame

In [ ]:
while(1):
    ser.write(hdlc_frame)
    ser.read_all()

In [ ]:
ser.write(hdlc_frame)
ser.read_all()

## Send sine signal in a Loop

In [ ]:
sine = create_sine_samples(amplitude=2000, frequency=500, samplerate=44092)
fig = pl.figure()
pl.plot(sine, figure=fig)

sine_len = len(sine)
# get free mem in fpga
response_size = 2

ser.reset_input_buffer()
ser.reset_output_buffer()

ser.write(bytes([0x7E]))
time.sleep(1)
rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)
sine_pos = 0


detect_underflow = False
#read data
while 1:
    print("Free mem:", free_mem)
    print("In buff:", ser.in_waiting)
    if (free_mem < 16000) and (detect_underflow == False):
        detect_underflow = True
    if (free_mem > 16000) and (detect_underflow == True):
        msg = "Potential Underflow!"
        raise serial.SerialException(f'{msg}')
    if free_mem > sine_len:
        # build payload
        start_fragment = sine_len - sine_pos
        remaining_len = free_mem - start_fragment
        frame_cnt, end_fragment = divmod(remaining_len, sine_len)
        payload = sine[sine_pos:] + frame_cnt * sine + sine[0:end_fragment]
        sine_pos = end_fragment

        # serialize frame
        frame = create_hdlc_frame(array_to_big_endian_bytes(np.array(payload)) )       
        
        ser.write(frame)
        time.sleep(1e-3)
        rec = ser.read(4)[-2:].hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)



In [ ]:
%debug


In [ ]:
fig = pl.figure()
pl.plot(payload, figure = fig)

In [ ]:
ser.in_waiting

In [ ]:
l = [0,1,2,3]
l[-2:]


# Open wav file

In [ ]:
wav_path = Path(r'..\test\Finjark_Master_003.wav')

In [9]:
wav_path = Path(r'..\test\Finjark_Master_003_mono.wav')

In [ ]:
wav_path = Path(r'..\test\440Hz_and_880Hz.wav')

In [ ]:
wav_path = Path(r'..\test\sine.wav')

In [ ]:
wav = wave.open(str(wav_path), mode='rb')

In [ ]:
wav.close()

## Just play the song direcly in Python

In [ ]:
p = pyaudio.PyAudio() 

wav = wave.open(str(wav_path), mode='rb')

print(wav.getsampwidth())
print(wav.getnchannels())
print(wav.getframerate())

#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True)  

print(stream)
print(type(stream))


#read data  
data = wav.readframes(160)
dt = np.dtype(np.int16) #.newbyteorder('>')
data = np.frombuffer(data, dtype=dt)
print(type(data))
print(data)
pl.plot(data)

#play stream  
while data:  
    stream.write(data)  
    data = wav.readframes(16000) 

wav.close()

## Send Data of song via UART to PC and play it in Python

In [ ]:
ser.close()
ser = serial.Serial(port='COM5', baudrate=256e3, timeout=5, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE)

In [ ]:
chunk_size = 20
data = None
wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 

ser.write(wav.readframes(chunk_size))
#data = ser.read_all()

#read data
#while data:
#    stream.write(data)
#    ser.write(wav.readframes(chunk_size))
#    data = ser.read_all()
    


#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

## Send bytes of wav and receive aknowledge

In [16]:
free_mem = 0
response_size = 2

p = pyaudio.PyAudio()  

wav = wave.open(str(wav_path), mode='rb')
audio_data = attenuate_audio(wav, divisor=16)
wav.close()
#fig = pl.figure()
#print(list(audio_data))
#raise Exception("deine mudda!")

#pl.plot(audio_data, figure=fig)
print("Audio file size:", len(audio_data))


ser.write(bytes([0x7E]))
rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)

detect_underflow = False
song_pos = 0

#read data
while 1:
    print("Free mem:", free_mem)
    if (free_mem < 10000) and (detect_underflow == False):
        detect_underflow = True
    if (free_mem > 16000) and (detect_underflow == True):
        msg = "Potential Underflow!"
        raise serial.SerialException(f'{msg}')
    if free_mem > 0:
        
        chunk = audio_data[song_pos : song_pos + free_mem]
        #pl.plot(chunk, figure=fig)
        
        frame = create_hdlc_frame(bytes(array_to_big_endian_bytes(chunk)))
        song_pos += free_mem
        ser.write(frame)

        rec = ser.read(4)[-2:].hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)

Audio file size: 15326225
Free mem: 16383
Free mem: 7238
Free mem: 3229
Free mem: 1490
Free mem: 717
Free mem: 363
Free mem: 195
Free mem: 103
Free mem: 91
Free mem: 86
Free mem: 84
Free mem: 92
Free mem: 86
Free mem: 85
Free mem: 87
Free mem: 90
Free mem: 91
Free mem: 86
Free mem: 87
Free mem: 93
Free mem: 85
Free mem: 86
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 89
Free mem: 89
Free mem: 87
Free mem: 89
Free mem: 88
Free mem: 94
Free mem: 85
Free mem: 85
Free mem: 86
Free mem: 111
Free mem: 79
Free mem: 94
Free mem: 76
Free mem: 79
Free mem: 88
Free mem: 93
Free mem: 90
Free mem: 92
Free mem: 84
Free mem: 84
Free mem: 88
Free mem: 91
Free mem: 90
Free mem: 92
Free mem: 88
Free mem: 83
Free mem: 85
Free mem: 89
Free mem: 92
Free mem: 90
Free mem: 85
Free mem: 88
Free mem: 93
Free mem: 85
Free mem: 84
Free mem: 92
Free mem: 88
Free mem: 89
Free mem: 85
Free mem: 87
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 9

Free mem: 85
Free mem: 86
Free mem: 88
Free mem: 89
Free mem: 91
Free mem: 89
Free mem: 87
Free mem: 86
Free mem: 91
Free mem: 86
Free mem: 88
Free mem: 87
Free mem: 93
Free mem: 86
Free mem: 84
Free mem: 88
Free mem: 88
Free mem: 90
Free mem: 89
Free mem: 88
Free mem: 90
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 85
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 93
Free mem: 90
Free mem: 82
Free mem: 102
Free mem: 85
Free mem: 79
Free mem: 87
Free mem: 91
Free mem: 90
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 86
Free mem: 89
Free mem: 87
Free mem: 87
Free mem: 89
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 87
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 94
Free mem: 90
Free mem: 85
Free mem: 90
Free mem: 92
Free mem: 84
Free mem: 88
Free mem: 90
Free mem: 85
Free mem: 86
Free mem: 89
Free mem: 89
Free mem: 8

Free mem: 89
Free mem: 89
Free mem: 87
Free mem: 95
Free mem: 84
Free mem: 83
Free mem: 88
Free mem: 93
Free mem: 90
Free mem: 86
Free mem: 86
Free mem: 87
Free mem: 91
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 93
Free mem: 86
Free mem: 84
Free mem: 88
Free mem: 90
Free mem: 89
Free mem: 87
Free mem: 88
Free mem: 93
Free mem: 86
Free mem: 84
Free mem: 86
Free mem: 91
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 86
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 87
Free mem: 92
Free mem: 86
Free mem: 86
Free mem: 88
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 93
Free mem: 87
Free mem: 85
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 89
Free mem: 88
Free mem: 86
Free mem: 89
Free mem: 89
Free mem: 87
Free mem: 90
Free mem: 87
Free mem: 88
Free mem: 87
Free mem: 88

Free mem: 39
Free mem: 54
Free mem: 41
Free mem: 36
Free mem: 43
Free mem: 48
Free mem: 53
Free mem: 47
Free mem: 41
Free mem: 41
Free mem: 45
Free mem: 46
Free mem: 44
Free mem: 35
Free mem: 40
Free mem: 50
Free mem: 49
Free mem: 39
Free mem: 40
Free mem: 45
Free mem: 45
Free mem: 47
Free mem: 42
Free mem: 45
Free mem: 47
Free mem: 42
Free mem: 41
Free mem: 46
Free mem: 46
Free mem: 41
Free mem: 42
Free mem: 45
Free mem: 45
Free mem: 45
Free mem: 43
Free mem: 48
Free mem: 42
Free mem: 42
Free mem: 43
Free mem: 45
Free mem: 45
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 49
Free mem: 46
Free mem: 43
Free mem: 41
Free mem: 43
Free mem: 44
Free mem: 45
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 45
Free mem: 44
Free mem: 43
Free mem: 43
Free mem: 41
Free mem: 47
Free mem: 47
Free mem: 44
Free mem: 39
Free mem: 42
Free mem: 45
Free mem: 46
Free mem: 44
Free mem: 43
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 46
Free mem: 43
Free mem: 44
Free mem: 45

Free mem: 45
Free mem: 51
Free mem: 46
Free mem: 40
Free mem: 45
Free mem: 43
Free mem: 42
Free mem: 41
Free mem: 48
Free mem: 48
Free mem: 44
Free mem: 39
Free mem: 41
Free mem: 43
Free mem: 47
Free mem: 45
Free mem: 43
Free mem: 45
Free mem: 42
Free mem: 43
Free mem: 46
Free mem: 46
Free mem: 44
Free mem: 42
Free mem: 43
Free mem: 44
Free mem: 43
Free mem: 43
Free mem: 50
Free mem: 42
Free mem: 42
Free mem: 42
Free mem: 44
Free mem: 45
Free mem: 52
Free mem: 44
Free mem: 37
Free mem: 40
Free mem: 46
Free mem: 47
Free mem: 44
Free mem: 43
Free mem: 44
Free mem: 44
Free mem: 45
Free mem: 45
Free mem: 44
Free mem: 43
Free mem: 43
Free mem: 45
Free mem: 45
Free mem: 43
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 45
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 49
Free mem: 46
Free mem: 39
Free mem: 41
Free mem: 44
Free mem: 46
Free mem: 45
Free mem: 43
Free mem: 44

...

Free mem: 49
Free mem: 46
Free mem: 42
Free mem: 42
Free mem: 47
Free mem: 48
Free mem: 42
Free mem: 41
Free mem: 40
Free mem: 44
Free mem: 49
Free mem: 47
Free mem: 42
Free mem: 39
Free mem: 49
Free mem: 48
Free mem: 42
Free mem: 42
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 46
Free mem: 43
Free mem: 43
Free mem: 44
Free mem: 41
Free mem: 44
Free mem: 46
Free mem: 46
Free mem: 43
Free mem: 43
Free mem: 42
Free mem: 47
Free mem: 45
Free mem: 41
Free mem: 45
Free mem: 43
Free mem: 46
Free mem: 43
Free mem: 61
Free mem: 78
Free mem: 51
Free mem: 77
Free mem: 54
Free mem: 38
Free mem: 34
Free mem: 45
Free mem: 47
Free mem: 44
Free mem: 43
Free mem: 46
Free mem: 51
Free mem: 39
Free mem: 38
Free mem: 44
Free mem: 47
Free mem: 45
Free mem: 43
Free mem: 46
Free mem: 42
Free mem: 45
Free mem: 42
Free mem: 42
Free mem: 45
Free mem: 45
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 46
Free mem: 47
Free mem: 45
Free mem: 44
Free mem: 42
Free mem: 40
Free mem: 43

Free mem: 88
Free mem: 96
Free mem: 89
Free mem: 87
Free mem: 85
Free mem: 90
Free mem: 88
Free mem: 84
Free mem: 86
Free mem: 89
Free mem: 90
Free mem: 88
Free mem: 88
Free mem: 95
Free mem: 96
Free mem: 83
Free mem: 77
Free mem: 85
Free mem: 93
Free mem: 92
Free mem: 91
Free mem: 85
Free mem: 89
Free mem: 86
Free mem: 87
Free mem: 87
Free mem: 87
Free mem: 89
Free mem: 89
Free mem: 90
Free mem: 82
Free mem: 85
Free mem: 89
Free mem: 94
Free mem: 88
Free mem: 87
Free mem: 85
Free mem: 90
Free mem: 89
Free mem: 87
Free mem: 86
Free mem: 92
Free mem: 92
Free mem: 83
Free mem: 85
Free mem: 89
Free mem: 94
Free mem: 86
Free mem: 84
Free mem: 87
Free mem: 90
Free mem: 89
Free mem: 88
Free mem: 88
Free mem: 87
Free mem: 93
Free mem: 90
Free mem: 84
Free mem: 86
Free mem: 88
Free mem: 88
Free mem: 88
Free mem: 93
Free mem: 85
Free mem: 90
Free mem: 86
Free mem: 89
Free mem: 92
Free mem: 86
Free mem: 83
Free mem: 89
Free mem: 92
Free mem: 91
Free mem: 83
Free mem: 86
Free mem: 89
Free mem: 90

Free mem: 44
Free mem: 48
Free mem: 39
Free mem: 44
Free mem: 49
Free mem: 39
Free mem: 40
Free mem: 47
Free mem: 48
Free mem: 41
Free mem: 42
Free mem: 43
Free mem: 49
Free mem: 43
Free mem: 47
Free mem: 45
Free mem: 39
Free mem: 45
Free mem: 43
Free mem: 46
Free mem: 42
Free mem: 50
Free mem: 46
Free mem: 38
Free mem: 40
Free mem: 44
Free mem: 46
Free mem: 45
Free mem: 46
Free mem: 47
Free mem: 40
Free mem: 41
Free mem: 45
Free mem: 45
Free mem: 42
Free mem: 48
Free mem: 50
Free mem: 38
Free mem: 44
Free mem: 44
Free mem: 41
Free mem: 44
Free mem: 51
Free mem: 44
Free mem: 37
Free mem: 41
Free mem: 50
Free mem: 43
Free mem: 48
Free mem: 44
Free mem: 40
Free mem: 40
Free mem: 44
Free mem: 46
Free mem: 45
Free mem: 44
Free mem: 44
Free mem: 44
Free mem: 52
Free mem: 44
Free mem: 39
Free mem: 43
Free mem: 46
Free mem: 45
Free mem: 46
Free mem: 44
Free mem: 38
Free mem: 41
Free mem: 46
Free mem: 48
Free mem: 43
Free mem: 44
Free mem: 42
Free mem: 47
Free mem: 43
Free mem: 41
Free mem: 44

Free mem: 87
Free mem: 88
Free mem: 87
Free mem: 90
Free mem: 89
Free mem: 87
Free mem: 86
Free mem: 89
Free mem: 88
Free mem: 90
Free mem: 85
Free mem: 90
Free mem: 90
Free mem: 87
Free mem: 87
Free mem: 90
Free mem: 85
Free mem: 86
Free mem: 95
Free mem: 89
Free mem: 86
Free mem: 85
Free mem: 94
Free mem: 87
Free mem: 84
Free mem: 87
Free mem: 91
Free mem: 88
Free mem: 91
Free mem: 83
Free mem: 92
Free mem: 86
Free mem: 86
Free mem: 91
Free mem: 89
Free mem: 88
Free mem: 87
Free mem: 88
Free mem: 94
Free mem: 86
Free mem: 84
Free mem: 86
Free mem: 90
Free mem: 89
Free mem: 91
Free mem: 89
Free mem: 85
Free mem: 85
Free mem: 88
Free mem: 91
Free mem: 89
Free mem: 87
Free mem: 83
Free mem: 86
Free mem: 88
Free mem: 95
Free mem: 93
Free mem: 84
Free mem: 87
Free mem: 87
Free mem: 84
Free mem: 94
Free mem: 92
Free mem: 87
Free mem: 85
Free mem: 88
Free mem: 90
Free mem: 88
Free mem: 86
Free mem: 90
Free mem: 89
Free mem: 88
Free mem: 86
Free mem: 89
Free mem: 89
Free mem: 88
Free mem: 88

KeyboardInterrupt: 

In [ ]:
audio_data[song_pos : song_pos + 2*free_mem]

In [ ]:
type(audio_data)

In [ ]:
for byte in range(0,0x100,1):
    ser.write(bytes([byte]))

In [ ]:
print(wav.getsampwidth())


# Get decoded mp3 data

# Check consistency of sent data

In [ ]:
data = []
with open(r'..\test\debug_data.csv', 'r') as IN:
    for line in IN.readlines():
        data.append(line.replace('\n', '').split(';'))

In [ ]:
df = pd.DataFrame(data).T \
                  .rename(columns={0:'channel_0', 1:'channel_1', 2:'channel_2', 3:'channel_3'})

In [ ]:
df.head()

In [ ]:
comp_send_receive(df.channel_0.dropna().astype(int).values, np.arange(start=0x30, stop=0x40, step=1))

In [ ]:
qstr = f"channel_0 == '0'"
df.query(qstr)

In [ ]:
%debug

In [ ]:
hex(16128)

# Debug

In [ ]:
a = b'\x7E\x7D'

In [ ]:
int(a.hex(), 16)

In [ ]:
a.replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E').hex()

In [ ]:
type(wav.readframes(chunk_size))

In [ ]:
ser.timeout

In [ ]:
(b'\x7E' + wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E') + b'\x7E').hex()

In [ ]:
ser.close()

In [ ]:
wav.

In [ ]:
a = np.array([1, 2, 0, 1, 2, 0, 1, 2, 0, 1])
b = np.array([0, 1, 2])
c = np.array([1, 2, 0, 1, 2, 0, 1, 2, 0, 2])

In [ ]:
comp_send_receive(rec_data=a, sent_data=b)

In [ ]:
np.where(c == 1)